## Fase 4: Limpieza de datos

Tenemos más de 200 columnas de las que sólo queremos las concernientes a sus datos de programación

Nos vamos a quedar con las siguientes columnas

* languages: la correspondientes a cada uno de los lenguajes soportados en codewars
* stats sociales: allies, clan, comments, followers, following, github, etc. 
* stats estadísticos: honor, last seen, leaderboard position, etc. 


In [1]:
# imports 

import pandas as pd
import numpy as np
import requests
import json
from bs4 import BeautifulSoup
from IPython.display import display
import time
import re

from funciones_scraping import get_languages

pd.options.display.max_columns = None

df = pd.read_csv('../output/codewar_users.csv', index_col=0)

def get_numeric_groups(x, groups=1): 
    try: 
        numbers = [int(number) for number in re.findall(r'\d+', x)]
    except: 
        numbers = [0] * groups
    else: 
        pass
    finally: 
        # return list if groups > 1 else return first element in list
        return numbers[:groups] if groups > 1 else numbers[0]
    
def get_highest_trained(x): 
    try: 
        res = x.split('(')[0].strip()
        return res
    except: 
        return x
    
def get_honor_percentile(x): 
    try: 
        res = x.replace('Top', '').replace('%', '').strip()
        return res
    except: 
        return x


/home/albertogcmr/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (22,27,220) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
# clean column names
df.columns = df.columns.str.replace(' ', '_').str.replace('.', '').str.lower()

# replace nan in numeric colmuns with 0
df._get_numeric_data().fillna(0, inplace=True)

# creates two new columns and delete the previous
df["authored_translations"], df["approved_translations"] = zip(*df["authored_translations"].map(lambda x: get_numeric_groups(x, groups=2)))

# creates new column and delete the previous
df['avg_rank'] = df['avg_rank'].apply(lambda x: get_numeric_groups(x, groups=1))

# creates new column and delete the previous
df['avg_satisfaction_rating'] = df['avg_satisfaction_rating'].apply(lambda x: get_numeric_groups(x, groups=1))

# creates new column and delete the previous
df['best_practice'] = df['best_practice'].apply(lambda x: get_numeric_groups(x, groups=1))

# creates new column and delete the previous
df['best_practice_solutions'] = df['best_practice_solutions'].str.replace(',', '').apply(lambda x: get_numeric_groups(x, groups=1))

# creates new column and delete the previous
df['clever_solutions'] = df['clever_solutions'].str.replace(',', '').apply(lambda x: get_numeric_groups(x, groups=1))

# creates two new columns and delete the previous
df["comments"], df["replies"] = zip(*df["comments"].map(lambda x: get_numeric_groups(x, groups=2)))

# creates two new columns and delete the previous
df["created"], df["beta"] = zip(*df["created"].map(lambda x: get_numeric_groups(x, groups=2)))

# cleans dates column
df['date'] = pd.to_datetime(df['date'])
df['ended_on'] = pd.to_datetime(df['ended_on'])
df['first_completed'] = pd.to_datetime(df['first_completed'])
df['last_completed'] = pd.to_datetime(df['last_completed'])

# creates new column and delete the previous
df['highest_trained'] = df['highest_trained'].apply(get_highest_trained)

# creates new column and delete the previous
df['honor'] = df['honor'].str.replace(',', '').apply(lambda x: get_numeric_groups(x, groups=1))

# creates new column and delete the previous
df['honor_percentile'] = df['honor_percentile'].apply(get_honor_percentile)

In [3]:
# creates two new columns and delete the previous
df["kumite"], df["started_kumite"] = zip(*df["kumite"].map(lambda x: get_numeric_groups(x, groups=2)))

ValueError: not enough values to unpack (expected 2, got 1)

In [5]:
df.kumite.value_counts()

0                    11597
1 (1 Started)          725
2 (2 Started)          291
1 (2 Started)          118
3 (3 Started)          115
                     ...  
18 (19 Started)          1
3 (9 Started)            1
61 (58 Started)          1
271 (301 Started)        1
30 (30 Started)          1
Name: kumite, Length: 291, dtype: int64

In [32]:

df.honor.unique()

array(['31,022', '5,902', '8,518', ..., '2,082', '16,721', '2,540'],
      dtype=object)

In [29]:
df['honor_percentile'].unique()

array(['0.01', '0.22', '0.13', ..., '41.08', '11.22', '51.75'],
      dtype=object)

In [10]:
len(df.date.unique())

88

In [11]:
len(pd.to_datetime(df['date']).unique())

88

### 4.0 Drop columnas con muchos nulos

Eliminamos las columnas con más de la mitad de sus registros a NaN (no incluyendo github ni linkedind)

In [ ]:
# Vemos los nulos. 
redes_sociales = ['linkedin', 'github']
columnas_nulas = [col for col in df.columns if df[col].isna().sum()>len(df)//2 and col not in redes_sociales]
len(columnas_nulas)

In [12]:
df.head()

,1_kyu,2_kyu,3_kyu,4_kyu,5_kyu,6_kyu,7_kyu,8_kyu,advanced_language_features,agda,algebra,algorithms,allies,angular,applied_computer_science,arguments,arithmetic,arrays,ascii,asynchronous,authored_kata,authored_translations,avg_rank,avg_satisfaction_rating,babel,basic_language_features,best_practice,best_practice_solutions,bf,big_integers,binary,binary_search_trees,bits,bitwise_operators,booleans,bugs,c,case/switch_statements,character_encodings,chars,ciphers,clan,classes,clever,clever_solutions,clojure,closures,coffeescript,collections,comments,completed_kata,completed_on_1st_attempt,computability_theory,computational_science,conditional_statements,contributed_kata,control_flow,coq,cpp,created,cryptography,crystal,csharp,current_streak,dart,data,data_conversion,data_structures,data_types,databases,date,dates/time,decimals,declarative_programming,decoding,decryption,design_patterns,design_principles,dictionary,dynamic_programming,elixir,elm,encoding,encryption,ended_on,erlang,es2015,esoteric_languages,event_handling,exception_handling,filtering,first_completed,followers,following,formats,formatting,fortran,frameworks,fsharp,functional_programming,functions,fundamentals,game_boards,games,geometry,github,go,graphics,graphs,groovy,hacking_holidays,hashes,haskell,higher-order_functions,highest_trained,honor,honor_percentile,idris,immutability,information_systems,inheritance,integers,interfaces,interpreters,interview_questions,io,iterators,java,javascript,json,julia,kata_approvals,kata_attempts,kotlin,kumite,lambdas,last_completed,last_seen,leaderboard_position,linked_lists,linkedin,lists,logic,loops,lua,machine_learning,map/reduce,maps,mathematics,member_since,memoization,metaprogramming,methods,modules,most_number_of_days,most_recent,mutability,name,nasm,networks,nim,nodejs,numbers,objc,object-oriented_programming,objects,observers,ocaml,optimization,parsing,performance,permutations,php,physics,polymorphism,powershell,programming_paradigms,properties,prototype-based_programming,prototypes,purescript,puzzles,python,queues,r,racket,rank,react,reason,recursion,refactoring,reflection,regular_expressions,reporting,ruby,rules,rust,scala,search,security,sequences,sets,shell,simulation,skills,social,solidity,sorting,sparse_arrays,sql,state_machines,statistics,streams,strings,swift,tables,testing,theorem_proving,theoretical_computer_science,total_collected,total_completed_kata,total_completions,total_languages_trained,total_stars,translations,trees,typescript,unicode,user,utilities,validation,variables,vectors,weak_typing,approved_translations,replies,beta
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,5,76,0.0,0.0,0,0,96.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1187.0,0.0,0.0,0.0,0.0,Unknown,0.0,0.0,0,0.0,0.0,0.0,0.0,5295,0.0,0.0,0.0,0.0,0.0,167.0,0.0,0.0,3342.0,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,56.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,https://github.com/FArekkusu,0.0,0.0,0.0,0.0,0.0,0.0,738.0,0.0,Python (1 kyu),"31,022",Top 0.01%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1384.0,12981.0,0.0,0.0,48.0,0.0,0.0,296 (276 Started),0.0,NaN,Jun 2019,#31,0.0,NaN,0.0,9.0,0.0,0.0,0.0,0.0,0.0,3.0,Sep 2017,0.0,0.0,0.0,0.0,0.0,C#,0.0,Alexander Fedorov,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18638.0,0.0,359.0,0.0,1 kyu,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4401.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,"{'dmivlge', 'Blue_Velvet', 'dandgerson', 'Drag...",0.0,0.0,0.0,707.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,48.0,"3,814",481.0,10.0,125.0,285 (239 approved),0.0,0.0,0.0,FArekkusu,0.0,0.0,0.0,0.0,0.0,0,1749,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Unknown,0.0,0.0,0,0.0,0.0,0.0,0.0,43,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,2450.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [ ]:
# hacemos drop de las columnas que tengan más del 50% a nulos

df.drop(columns=columnas_nulas, inplace=True)
df.columns

### 4.1 Clan

No vamos a cambiar el valor de los Unknown de esta columna porque lo mismo es "" que 'Unknown'

In [ ]:
# Esta columna no aporta demasiados datos debido a su baja variabilidad
# Vamos a eliminarla
df.clan.value_counts().head()
df.drop(columns=['clan'], inplace=True)

### 4.2 Collections

In [ ]:
df.collections.value_counts()

# No hay que limpiar nada

### 4.3 Comments

In [ ]:
df.comments.value_counts().head(10)

In [ ]:
def clean_comments(x): 
    try: 
        res = x.split(' ')[0]
        return float(res)
    except: 
        return 0

In [ ]:
# df['comments'] = df['comments'].apply(lambda x: clean_comments(x))
df['comments'] = df['comments'].apply(clean_comments)

df.comments.value_counts().head(10)

### 4.4 highest trained

In [ ]:
# Vamos a ver si merece la pena highest trained o no. si el lenguaje coincide con su score más alto no merece la pena

# df.iloc[44]
# Podemos hacer drop a la columna

df.drop(columns=['highest trained'], inplace=True)
df.columns

### 4.5 Honor | total completed kata

In [ ]:
def clean_str_float(x): 
    try: 
        return float(x.replace(',', ''))
    except: 
        return 0

In [ ]:
df['honor'] = df['honor'].apply(clean_str_float)
df['honor'].value_counts().head(20)

In [ ]:
df['honor'].max()

In [ ]:
df['total completed kata'] = df['total completed kata'].apply(clean_str_float)
df['total completed kata'].value_counts().head(10)

### 4.6 honor percentile | most recent | name

* Honor percentile es un valor calculado a partir de los niveles de score de sus katas, podemos eliminarlo sin problemas. 
* most recent lo utilizaremos para nuestro modelo de entrenamiento supervisado
* name no nos interesa

In [ ]:
columns_to_drop = ['honor percentile', 'name']

df.drop(columns=columns_to_drop, inplace=True)
df.columns

### 4.7 kumite | translations

In [ ]:
# df['kumite'] = df['kumite'].apply(lambda x: clean_honor(x))
def remove_parentesis(x): 
    try: 
        res = float(x.split(' ')[0])
        return res
    except: 
        return 0

In [ ]:
df['kumite'] = df['kumite'].apply(remove_parentesis)
df['kumite'].value_counts().head(15)

In [ ]:
df['translations'] = df['translations'].apply(remove_parentesis)
df['translations'].value_counts().head(10)

### 4.8 last seen | member since

Convertimos estas columnas a tipo Datetime para su mejor tratamiento

In [ ]:
df['member since'] = pd.to_datetime(df['member since'])
df['last seen'] = pd.to_datetime(df['last seen'])

### 4.9 Github y linkedin

Rellenamos los huecos vacios de estas columnas con strings vacios. 

In [ ]:
df['linkedin'] = df['linkedin'].fillna('')
df['github'] = df['github'].fillna('')

In [ ]:
df['github'].value_counts().head(), df['linkedin'].value_counts().head()

Porcentaje de usuarios con github

In [ ]:
len(df[df['github'] != ''])/len(df)

Porcentaje de usuarios con linkedin

In [ ]:
len(df[df['linkedin'] != ''])/len(df)

### 4.10 leaderboard position

In [ ]:
def clean_leaderboard_position(x): 
    try: 
        res = float(x.replace('#', '').split(' ')[0])
        return res
    except: 
        return 0
df['leaderboard position'] = df['leaderboard position'].apply(clean_leaderboard_position)
# df['leaderboard position'].value_counts().head(10)

### 4.11 rank

In [ ]:
def clean_rank(x): 
    try: 
        is_dan = 'dan' in x
        res = float(x.split(' ')[0])
        return res if not is_dan else -res
    except: 
        return 0

In [ ]:
df['rank'] = df['rank'].apply(clean_rank)
df['rank'].value_counts().head(25)

In [ ]:
df.shape

In [ ]:
nulos = {col: df[col].isna().sum() for col in df.columns}
nulos

In [ ]:
# Los nulos se agrupa de 46 en 46 así que voy a suponer y comprobar que son 46 registros inválidos

df.dropna(inplace=True)

nulos = {col: df[col].isna().sum() for col in df.columns}
nulos

In [ ]:
df.shape

### 4.12 Member since | Last Seen

In [ ]:
# Vamos a crear una columna honor density = honor/time
# Hacer drop de las originales

In [ ]:
# df.iloc[4]['member since'].year

df['days of activity'] = (df['last seen'] - df['member since']).dt.days
# df.drop(columns=['last seen', 'member since'], inplace=True)

In [ ]:
df.dtypes

### 4.13 Guardamos los datos

In [ ]:
df.to_csv('../output/df-limpio-iter3.csv')
df.shape

### Resultado

Hemos generado un CSV con registros y columnas (11500x57) que hemos limpiado. Todos salvo 4 son te tipo numérico lo que nos ayudará a análisis posteriores. 
Las columnas no numéricas son: 
* username: nos permitirá identificar al usuario
* linkedin: permite su acceso por las redes sociales y por la API propia. 
* github: permite su acceso por la API propia. 
* most recent: Permitirá ejecutar algoritmos de ML supervisado para intentar predecir este valor

In [ ]:
df.head()